# Read in the images (X) and their hair color labels (Y)

In [16]:
#Download image files
!gdown  --id 1ahOw9goNWSZKF4OdWzsMtiCL772ATT2H
!tar -xf holmes_red_headed_league_data.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 66714    0 66714    0     0   119k      0 --:--:-- --:--:-- --:--:--  118k


In [12]:
from skimage import io as skimage_io
from skimage.transform import resize
import os, io, glob
from IPython.display import Image, display
from IPython.core.display import display, HTML

images =[]
labels = []


# these hair color pics were downloaded from IMDB's list of actresses that were
# "blonde", "brunette", and "red heads"
# example https://www.imdb.com/list/ls058386015/  -- a list of blonde actresses
folders = ["data/blonde/blonde*jpg", "data/brunette/brown*jpg", "data/redheads/red*jpg"]

# assign jpg_files all of the hair-colored jpgs
jpg_files = []
for folder in folders:
    jpg_files.extend (glob.glob (folder))



IMG_PX_SIZE = 20 # pixels. All pictures will be resized to IMG_PX_SIZE x IMG_PX_SIZE


for jpg_file in jpg_files:
    
    # assign the hair color of this image to label.
    # e.g. filename "red1.jpg" becomes red, "brown2.jpg" becomes brown
    label = os.path.basename (jpg_file).replace (".jpg", "")
    label = ''.join([i for i in label if not i.isdigit()]) 

    # read the image into rgb_img
    rgb_img = skimage_io.imread (jpg_file)
    
    # resize the image, so all images have the same size
    rgb_img = resize (rgb_img, (IMG_PX_SIZE,IMG_PX_SIZE))

    # store the picture information in X, and the hair color in Y
    # we will try to predict Y based on X (predict hair color based on a picture)
    images.append (rgb_img.reshape (1, IMG_PX_SIZE*IMG_PX_SIZE*3)[0]) # flatten into a 1-D array
    labels.append (label)
    
    display (HTML(label))
    display(Image(filename=jpg_file))
    
print ("read in " + str (len(images)) + " images")

read in 33 images


# Create a classifier

This is a machine learning method that creates groups based on the labels we provide it. In this case, we are creating 3 groups (blonde, red, brunette). The model will learn from the blonde, red, brunette pictures (X) and will make a classifier based on the labels provided (Y). 


In [30]:
from sklearn.neighbors import KNeighborsClassifier

num_groups = 3  # blonde, red, brunette

# initialize the model with KNeighborsClassifier and the number of groups
hair_classifer = KNeighborsClassifier(n_neighbors=num_groups)

# make the classifier based on X (pictures) and Y (their hair color labels)
hair_classifier.fit (images, labels)



KNeighborsClassifier(n_neighbors=3)

# Calculate classifier's performance

Given an image (X), what does the model predict
Compare the model's predictions with the true labels (Y)

In [31]:
from sklearn.metrics import accuracy_score

predicted_labels = hair_classifier.predict (images)

# get accuracy of K-neighbors classifier
accuracy_score (labels,predicted_labels )

0.9393939393939394

The classifier is 94% accurate.

# Visualization of the Classifier

In [ ]:
from mlxtend.plotting import plot_decision_regions
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

# turn red, blonde, brown hair labels into integers so sklearn can read it
label_encoder = LabelEncoder()
label_encoder.fit (labels)
Y_encoded = label_encoder.transform(labels)
le_name_mapping = dict(zip(label_encoder.transform(label_encoder.classes_), label_encoder.classes_))

# for plotting, get the 2 most important PCA components
pca = PCA(n_components = 2)
X_2 = pca.fit_transform(images)
vis_model = model.fit(X_2, Y_encoded)

#clf is the classifier object
plot_decision_regions(X_2, Y_encoded, clf=vis_model, legend=2)

handles, labels =  plt.gca().get_legend_handles_labels()
print (list(map(le_name_mapping.get, [int(i) for i in le_name_mapping])))
plt.legend(handles, list(map(le_name_mapping.get, [int(i) for i in le_name_mapping])))


plt.xlabel("PCA1", size=14)
plt.ylabel("PCA2", size=14)
plt.title("K Neighbors Classifier with 3 Groups (K=3)")
plt.show()

# Classify Wilson's hair

Put the image (wilsons_hair.jpg) in exactly the same format as the other pictures so we can feed it into the classifier


## Read in the image of Wilson's hair and make it the same size as the other images

In [33]:
# read Wilson's hair

from skimage import io as skimage_io
from skimage.transform import resize

jpg_file = "data/wilsons_hair.jpg"

# read picture into rgb_img
display(Image(filename=jpg_file))
rgb_img = skimage_io.imread (jpg_file)
    
# resize the image to same as the other jpg's used to create the classifier

rgb_img = resize (rgb_img, (IMG_PX_SIZE,IMG_PX_SIZE))

# store the picture information in X
# flatten the 3 channels red, green, blue into a 1-D array with the reshape command
wilsons_hair = []
wilsons_hair.append (rgb_img.reshape (1, IMG_PX_SIZE*IMG_PX_SIZE*3)[0])



## Classify Wilson's hair color, and look at the probability / confidence of this classification

In [ ]:
prediction = hair_classifier.predict (wilsons_hair)
print ("Predicted hair color")
print (prediction[0])

import pandas as pd
prob_results = pd.DataFrame (hair_classifier.predict_proba (wilsons_hair), columns = hair_classifier.classes_)
print ("\nProbability")
print (output.to_string (index=False))